In [1]:
with open('wiz_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
79


In [2]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))


[60, 61, 1, 72, 60, 57, 70, 57]
hi there


In [3]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([226639]) torch.int64
tensor([ 0,  0, 26, 31, 24, 39, 43, 28, 41,  1, 12, 10,  0,  0, 43, 31, 28,  1,
        28, 24, 41, 43, 31, 40, 44, 24, 34, 28,  0,  0,  0, 43, 60, 57,  1, 72,
        70, 53, 61, 66,  1, 58, 70, 67, 65,  1,  5, 29, 70, 61, 71, 55, 67,  1,
        75, 53, 71,  1, 74, 57, 70, 77,  1, 64, 53, 72, 57, 10,  1, 32, 72,  1,
        71, 60, 67, 73, 64, 56,  1, 60, 53, 74, 57,  1, 53, 70, 70, 61, 74, 57,
        56,  1, 53, 72,  1, 31, 73, 59, 71, 67, 66,  5, 71,  0, 71, 61, 56, 61,
        66, 59,  1, 53, 72,  1, 65, 61, 56, 66, 61, 59, 60, 72,  8,  1, 54, 73,
        72,  1, 61, 72,  1, 75, 53, 71,  1, 53, 64, 70, 57, 53, 56, 77,  1, 58,
        61, 74, 57,  1, 67,  5, 55, 64, 67, 55, 63,  1, 53, 66, 56,  1, 72, 60,
        57,  1, 59, 70, 53, 77,  1, 56, 53, 75, 66,  0, 75, 53, 71,  1, 54, 70,
        57, 53, 63, 61, 66, 59,  1, 61, 66,  1, 72, 60, 57,  1, 57, 53, 71, 72,
         1, 75, 60, 57, 66,  1, 72, 60, 57,  1, 64, 61, 72, 72, 64, 57,  1, 72,
       

In [4]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [5]:
block_size=8
train_data[:block_size+1]

tensor([ 0,  0, 26, 31, 24, 39, 43, 28, 41])

In [6]:
x= train_data[:block_size]
y= train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when context is {context} the target is {target}:")

when context is tensor([0]) the target is 0:
when context is tensor([0, 0]) the target is 26:
when context is tensor([ 0,  0, 26]) the target is 31:
when context is tensor([ 0,  0, 26, 31]) the target is 24:
when context is tensor([ 0,  0, 26, 31, 24]) the target is 39:
when context is tensor([ 0,  0, 26, 31, 24, 39]) the target is 43:
when context is tensor([ 0,  0, 26, 31, 24, 39, 43]) the target is 28:
when context is tensor([ 0,  0, 26, 31, 24, 39, 43, 28]) the target is 41:


In [7]:
torch.manual_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
    data=train_data if split =='train' else val_data
    ix= torch.randint(len(data) - block_size, (batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size):
    for t in range(block_size):
        context=xb[b,:t+1]
        target=yb[b,t]
        print(f"when context is {context} the target : {target}:")

inputs:
torch.Size([4, 8])
tensor([[57, 66,  8,  1, 72, 60, 67, 73],
        [ 1, 71, 75, 57, 57, 72,  1, 65],
        [57,  8,  1, 54, 57, 55, 53, 73],
        [71,  8,  1, 53, 66, 56,  1, 33]])
targets:
torch.Size([4, 8])
tensor([[66,  8,  1, 72, 60, 67, 73, 59],
        [71, 75, 57, 57, 72,  1, 65, 73],
        [ 8,  1, 54, 57, 55, 53, 73, 71],
        [ 8,  1, 53, 66, 56,  1, 33, 61]])
------
when context is tensor([57]) the target : 66:
when context is tensor([57, 66]) the target : 8:
when context is tensor([57, 66,  8]) the target : 1:
when context is tensor([57, 66,  8,  1]) the target : 72:
when context is tensor([57, 66,  8,  1, 72]) the target : 60:
when context is tensor([57, 66,  8,  1, 72, 60]) the target : 67:
when context is tensor([57, 66,  8,  1, 72, 60, 67]) the target : 73:
when context is tensor([57, 66,  8,  1, 72, 60, 67, 73]) the target : 59:
when context is tensor([1]) the target : 71:
when context is tensor([ 1, 71]) the target : 75:
when context is tensor([ 1,

In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 79])
tensor(4.7223, grad_fn=<NllLossBackward0>)

D.'CHWmYgV6CiL4Q8k]MKC,?&pSZ2RvV3_rgj-3wI:
HE-sTbK?'Sd7xk-T1c5n:u,._ykK?2PQYVIs]o,k nZ!U5fY0vDH4bhcK


In [9]:
optimizer=torch.optim.Adam(m.parameters(),lr=1e-5)

In [10]:
batch_size = 32
for steps in range(5000): 
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())



4.64561128616333


In [11]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



YDR
blvYM,4OVCY85Amv
lW0ILqC2,.)6"83zMz6leF6A bHhdX(Nu,J0pR_Eg -to,JN2:?epKNLoKLfN
iv_EqCHLo'C,z&&Rd:evV
i -ANz7EdXFe[SdswQLqkp(:9qw?7aHHxMmvyGyUY'Cm)nuEOzMh1.D
8Yh,zZlHJ2Msk2x9DHU_aLWEkYLP;'Jdhy_U8r06k5z7(fBv(dmyigX
GdaSzZ wqJs"8FS]Fa"8&R2n'EScEu55Al CmQ[R.D!R&wAl1C-Jn
Y[f-K
hdAFJqJ2x1918aXZ1-"xzC6


In [12]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [79]:
from sklearn.linear_model import LinearRegression
import numpy as np
import torch
import torch.nn as nn

with open('wiz_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size=len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

X = data[:-1].unsqueeze(1)
y = data[1:]

model = LinearRegression()
model.fit(X, y)

next_numbers = []
sequence = X[-1][0]
for _ in range(10):
    next_number = model.predict([[sequence]])
    next_numbers.append(next_number[0])
    sequence += 1

print("Next 10 numbers in the sequence:", next_numbers)


Next 10 numbers in the sequence: [50.65351319789421, 50.61359242346338, 50.57367164903255, 50.533750874601715, 50.49383010017089, 50.45390932574006, 50.413988551309224, 50.37406777687839, 50.33414700244756, 50.294226228016726]


In [78]:
import numpy as np
import torch
import torch.nn as nn

def generate_next_tokens(model, start_sequence, max_new_tokens):
    start_sequence_encoded = torch.tensor(encode(start_sequence), dtype=torch.long).unsqueeze(0)
    new_sequence = start_sequence_encoded.clone()
    sequence = start_sequence_encoded
    hidden = None
    for _ in range(max_new_tokens):
        output, hidden = model(sequence, hidden)
        sequence = torch.argmax(output, dim=2)
        new_sequence = torch.cat((new_sequence, sequence), dim=1)
    new_tokens = [itos[i.item()] for i in new_sequence.squeeze()]
    return ''.join(new_tokens)

with open('wiz_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size=len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        x, hidden = self.rnn(x, hidden)
        x = self.fc(x)
        return x, hidden

X = data[:-1][:, np.newaxis]
y = data[1:]

X_tensor = X.clone().detach().long()
y_tensor = y.clone().detach().long()

model = SimpleRNN(vocab_size, 128, vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    optimizer.zero_grad()
    outputs, _ = model(X_tensor, None)
    loss = criterion(outputs.view(-1, vocab_size), y_tensor.view(-1))
    loss.backward()
    optimizer.step()

generated_sequence = generate_next_tokens(model, '1', 200)
print("Generated Sequence:", generated_sequence)


Generated Sequence: 16 the an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an


In [77]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam

def get_batch(text, seq_length, batch_size):
    starts = np.random.randint(0, len(text) - seq_length - 1, batch_size)
    sequences = [text[start:start + seq_length] for start in starts]
    targets = [text[start + 1:start + seq_length + 1] for start in starts]
    X = torch.tensor([item for sublist in sequences for item in sublist], dtype=torch.long).view(batch_size, seq_length)
    y = torch.tensor([item for sublist in targets for item in sublist], dtype=torch.long).view(batch_size, seq_length)
    return X, y

def generate_next_tokens(model, start_sequence, max_new_tokens):
    start_sequence_encoded = torch.tensor(encode(start_sequence), dtype=torch.long).unsqueeze(0)
    new_sequence = start_sequence_encoded.clone()
    sequence = start_sequence_encoded
    hidden = None
    for _ in range(max_new_tokens):
        output, hidden = model(sequence, hidden)
        probabilities = F.softmax(output, dim=2)
        sequence = torch.multinomial(probabilities.squeeze(), 1).unsqueeze(0)
        new_sequence = torch.cat((new_sequence, sequence), dim=1)
    new_tokens = [itos[i.item()] for i in new_sequence.squeeze()]
    return ''.join(new_tokens)

with open('wiz_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        x, hidden = self.rnn(x, hidden)
        x = self.fc(x)
        return x, hidden

seq_length = 50
batch_size = 64
hidden_size = 128
num_epochs = 100

data = torch.tensor(encode(text), dtype=torch.long)

model = SimpleRNN(vocab_size, hidden_size, vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    X, y = get_batch(data, seq_length, batch_size)
    outputs, _ = model(X, None)
    loss = criterion(outputs.view(-1, vocab_size), y.view(-1))
    loss.backward()
    optimizer.step()

generated_sequence = generate_next_tokens(model, '1', 2000)
print("Generated Sequence:", generated_sequence)


Generated Sequence: 1W"Tand, treroucand the tofer anjdelneve:rs cighatnon the sebres them cor sle sowed theattirvthe pory,"
slo, eir theint sa'd garore se ar7at them ancerenbwned buonleaagheg Wacher slicg hay wec'9d creedey f mane g si, Wqug sere dir ane wourd my ancer ald he ssthin gu.
In,urtt dal antr ullat, wire he ilsth, tac, seme uly toou oroobedun th min band sware he worout linz-ly.

folos some
det the i'K sineng" an wemn unmor ond yever ane ove reder now heand ulathe the, sahy wis the?n, Aescou bilf acleE, ard oftard aporoe shat sowit, easlist in The the pla Jashy hhemtintcar Wi giz?bto.
"Wound are.

"O
Ithim sare to tped the the thine a[d wou the tterpee

igr 6jos hitgneam of,frit pead tithesing re. II sichat warkg, I ther""

"[ur-verarnwir ofdkp jint ilag suate rarp"(latle rewkikc pilonTt tisy gardoll
"n roth "
oog tht eacedond tinrelloungyys hoPg
fow leranl to thapinP zand peiathe wand athed caro herdficingsthe se tired en jrUkao ait "I the in, youlumac Ther them Bfollm Eo 0

In [85]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('wiz_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l]) 

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        wei = q @ k.transpose(-2,-1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v 
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x) 
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
            idx_next = torch.multinomial(probs, num_samples=1) 
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx

model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.211535 M parameters
step 0: train loss 4.5662, val loss 4.5715
step 100: train loss 2.6259, val loss 2.6892
step 200: train loss 2.4736, val loss 2.5397
step 300: train loss 2.3653, val loss 2.4257
step 400: train loss 2.2730, val loss 2.3437
step 500: train loss 2.2130, val loss 2.2803
step 600: train loss 2.1561, val loss 2.2290
step 700: train loss 2.0992, val loss 2.1824
step 800: train loss 2.0386, val loss 2.1075
step 900: train loss 1.9808, val loss 2.0813
step 1000: train loss 1.9477, val loss 2.0506
step 1100: train loss 1.9056, val loss 1.9940
step 1200: train loss 1.8714, val loss 1.9554
step 1300: train loss 1.8444, val loss 1.9326
step 1400: train loss 1.7988, val loss 1.9121
step 1500: train loss 1.7812, val loss 1.8858
step 1600: train loss 1.7570, val loss 1.8745
step 1700: train loss 1.7465, val loss 1.8486
step 1800: train loss 1.7320, val loss 1.8465
step 1900: train loss 1.6937, val loss 1.8139
step 2000: train loss 1.6896, val loss 1.8083
step 2100: train loss 1.